# Algorithme des *k* plus proches voisins
Le lien vers [openclassroom](https://openclassrooms.com/fr/courses/4011851-initiez-vous-au-machine-learning/4022441-tp-entrainez-le-modele-des-k-plus-proches-voisins-k-nn)

Celui vers le [TP de l'université de Lille donné dans moodle](http://www.grappa.univ-lille3.fr/~ppreux/ensg/miashs/l3-ap/tps/kppv/)

En guise d'initiation au problème d'apprentissage supervisé, nous allons implanter et manipuler l'algorithme des *k* plus proches voisins. 
L'objectif de ce TP est de faire reconnaître automatiquement par l'ordinateur des chiffres manuscrits (pour lire des chèques par exemple).

## 1. Lecture du jeux d'exemples

Avant même d'implanter l'algorithme des *k* plus proches voisins, nous avons besoin d'exemples qui seront traités par l'algorithme. Aussi, commençons par lire un jeu de données.

Il s'agit d'un jeu de données très célèbre appelé MNIST. Il est constitué d'un ensemble de 70000 images, 28x28 pixels, en noir et blanc annoté du chiffre correspondant (entre 0 et 9). Ce jeu utilise des données réelles qui ont déjà été pré-traitées pour être plus facilement utilisables par un algorithme.

![Un extrait du type d'images que l'on trouve dans le dataset MNIST](extraitMNIST.png)

Le jeu de données est relativement petit mais pour l'algorithme des *k* plus proches voisins, il est déjà trop gros pour obtenir rapidement des résultats. On va donc effectuer un échantillonnage et travailler sur seulement 5000 données.

In [7]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [8]:
print(mnist.data.shape)
print(mnist.target.shape)

(70000, 784)
(70000,)


In [14]:
import numpy as np

sample = np.random.randint(70000, size=5000)

data = mnist.data[sample]
#data = map(lambda x: map(int,x),data)

target = mnist.target[sample]
#target = map(int,target)

On obtient deux listes, **data** et **target**.
<ul>
<li> data contient les images sous forme d'une liste 28*28 = 784 entiers compris entre 0 et 255 correspondant aux différentes nuances de gris (0 étant blanc et 255 noir)</li>
<li> target contient les entiers correspondant à l'image</li>
</ul>

## 2. Déterminer le plus proche voisin d'un point
### Distance entre deux images
Avant toute chose, il nous faut définir une fonction qui mesure la distance entre deux images. Pour faire les choses les plus simples possibles pour l'instant, définissons une fonction distance qui prend deux images (deux listes de nombres) en paramètre et renvoie la somme des différences de chaque pixel (en valeur absolue).

In [10]:
def distance (a, b):
    somme = 0
    for i in range(784):
        somme += abs(a [i] - b [i])
    return (somme)

In [15]:
image1 = data['569']
image2 = data['65']
distance(image1,image2)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Le plus proche voisin
Écrire une fonction lePlusProcheVoisin qui prend en paramètre une image (une liste de 784 entiers compris entre 0 et 255) et renvoie l'indice dans data du plus proche voisin.

In [ ]:
def lePlusProcheVoisin (x):
    lePlusPres = 0
    distanceMin = float("inf")
    for i in range(len(data)):
        di = distance (x, data[i])
        if di != 0 and di < distanceMin:
            lePlusPres = i
            distanceMin = di
    return (lePlusPres)

In [ ]:
indice = lePlusProcheVoisin(image1)
print(indice)
print(target[indice])
print(target[569])

## 3. Déterminer les *k* plus proches voisins d'un point
Écrire une fonction lesKplusProchesVoisins qui prend en paramètre une image et une valeur de k et renvoie la liste des indices dans data des k plus proches voisins.
Quand vous prenez k = 1, cette fonction doit renvoyer le même résultat que la précédente, mis à part le fait que lePlusProcheVoisin renvoie une valeur numérique alors que lesKplusProchesVoisins renvoie une liste d'un élément.

In [ ]:
def indiceInsertion(listeKppv,element):
    debut = 0
    fin = len(listeKppv)-1
    m = (fin+debut)//2
    while(fin-debut>1):
        m = (fin+debut)//2
        if(element<listeKppv[m][1]):
            fin = m
        else:
            debut = m
    return m if element<listeKppv[m][1] else m+1

def lesKplusProchesVoisins (x, k):
    listeDesDistances = []
    for image in data:
        listeDesDistances.append (distance (x, image))
    Kppv = [(0,listeDesDistances[0])]
    for i in range (1,k):
        Kppv.insert(indiceInsertion(Kppv,listeDesDistances[i]),(i,listeDesDistances[i]))
    for i in range (k,len (data)):
        if listeDesDistances [i] != 0 and listeDesDistances [i] < Kppv[3][1] :
            Kppv.insert(indiceInsertion(Kppv,listeDesDistances[i]),(i,listeDesDistances[i]))
            Kppv.pop()
    return [couple[0] for couple in Kppv]

In [ ]:
Kppv = lesKplusProchesVoisins (image1, 5)
for indice in Kppv:
    print(target[indice])

In [ ]:
l=[1,2,5,6,7,24,46]
indice = [3,6]
print([l[ind] for ind in indice])

## 4. Prédire l'étiquette d'une donnée

Écrire une fonction *predire* qui prend en paramètre une image dans le même format que celles de data et un entier *k* et retourne le chiffre qui est prédit, c'est à dire le chiffre qui est supposé être représenté sur l'iamge.
On décide du chiffre représenté sur l'image en appliquant un choix à la majorité, à savoir le chiffre qui apparaît majoritairement sur les *k* plus proches voisins.

In [ ]:
def predire (l,k):
    Kppv = lesKplusProchesVoisins(l,k)
    decomptes = [0]*10
    for indice in Kppv:
        decomptes[target[indice]] += 1
    plusGrandDecompte = decomptes [0]
    for i in range (1,10):
        if decomptes [i] > plusGrandDecompte:
                plusGrandDecompte = decomptes [i]
                indice = i
    return (indice)


In [ ]:
predire(image1,5)

## 5. Optimisation

In [ ]:
from data_target_test import data as data_test,target as target_test

def taux_erreur(k):
    '''
    calcule le taux d'erreur avec la valeur k pour les k plus proches voisins
    '''
    t=0
    n=len(target_test)
    for i in range(n):
        if predire(target[i],k)!=target_test[i]:
            t+=1
    return t/n

def optimisation(n):
    '''
    détermine quelle valeur de k donne la meilleure prédiction, avec k entre 1 et n
    '''
    liste_taux=[taux_erreur(k) for k in range(1,n+1)]
    return liste_taux.index(min(liste_taux))+1

In [ ]:
taux_erreur(5)